In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('dark_background')
import seaborn as sns
from tqdm import tqdm

import cv2
import os
import tensorflow as tf
import sklearn as sk

import warnings
warnings.filterwarnings('ignore')

print('Using Tensorflow version:', tf.__version__)

!pip install keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from keras.utils import np_utils

from sklearn.metrics import classification_report

In [ ]:
main_data = pd.read_csv('data_labels_mainData.csv')
extra_data = pd.read_csv('data_labels_extraData.csv')

In [ ]:
main_data

In [ ]:
extra_data

## Predicting cellType

### Prepare train, test, validation dataset

In [ ]:
from sklearn.model_selection import train_test_split
X = main_data['ImageName']
Y = main_data['cellType']

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.33, random_state=99)
trainX, valX, trainY, valY = train_test_split(trainX, trainY, test_size=0.2, random_state=99)

trainY = trainY.ravel()
testY = testY.ravel()
valY = valY.ravel()

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(valX.shape)
print(valY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
trainX_img_list = trainX.tolist()
testX_img_list = testX.tolist()
valX_img_list = valX.tolist()

In [ ]:
def create_img_dataset(directory, img_list):
    IMAGE_SIZE = 27
    train_data = []
    for img in tqdm(img_list, desc='Extracting Image Progress'):
        path = os.path.join(directory, img)
        img = cv2.resize(cv2.imread(path), (IMAGE_SIZE,IMAGE_SIZE))
        train_data.append(np.array(img))
    print("Successfully import images!")
    # Convert to np.array and normalize pixel values to be between 0 and 1 
    return np.array(train_data).astype('float32') / 255.0 

In [ ]:
# CHANGE THE PATH TO IMAGE FOLDER IN YOUR LOCAL DEVICE!
# Khang desktop: C:\Users\User\Desktop\RMIT\Cloud - GitHub\Minh\ML\Asm\ASM2\patch_images
# Khang laptop: C:\Users\Laptop\Desktop\RMIT\Cloud-GitHub\Minh\ML\Asm\ASM2\patch_images
path = r"C:\Users\User\Desktop\RMIT\Cloud - GitHub\Minh\ML\Asm\ASM2\patch_images"

trainX_img = create_img_dataset(path, trainX_img_list)
testX_img = create_img_dataset(path, testX_img_list)
valX_img = create_img_dataset(path, valX_img_list)

In [ ]:
plt.imshow(trainX_img[0])

In [ ]:
plt.imshow(testX_img[0])

In [ ]:
plt.imshow(valX_img[0])

### Build model

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=trainX_img.shape[1:], activation='relu', padding='same'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))

## Simple CNN

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding='same', activation='relu', input_shape=trainX_img.shape[1:]))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(4, activation="softmax"))

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(lr=0.000001)
model.compile(optimizer = opt,
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['accuracy'])
model.summary()

In [ ]:
# Data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(featurewise_center=True,
                                   featurewise_std_normalization=True, 
                                   rotation_range=20, 
                                   shear_range=0.2, 
                                   zoom_range=0.2, 
                                   horizontal_flip=True, 
                                   fill_mode='nearest', 
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2)

train_it = train_datagen.flow(trainX_img, trainY)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
early = EarlyStopping(monitor='accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

hist = model.fit(trainX_img, trainY, 
                 validation_data = (valX_img, valY),
                 batch_size = 32, 
                 epochs = 200, 
                 verbose=1,
                 callbacks=[early,checkpoint])

In [ ]:
# Summarize history for accuracy
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# Summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
predictions = model.predict_classes(testX_img)
predictions = predictions.reshape(1,-1)[0]

print(classification_report(testY, predictions, target_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3']))